Idea:

Hacer Gaussian Process y Partial Pooling

Vamos a arrancar modelando como evoluciona el precio de un jugador cuando envejece. Para esto usaremos solo 2 parametros de nuestros datos:
- Edad del jugador
- Precio del jugador

Usaremos el Gaussian Process porque parece ser una distr que estima bien el comportamiento esperado.

Usaremos Partial Pooling para agrupar jugadores parecidos entre ellos (ver si conviene por continente, por liga, division, etc)

Vamos a agarrar un grupo de jugadores y generar el Gaussian Process entre ellos. Y asi con todos los conjuntos de grupos qu tengamos.


Link al dataset: https://www.kaggle.com/datasets/davidcariboo/player-scores/data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
from scipy.stats import expon, norm   # Medidas de desempeño
import pymc as pm

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Chequear que este en el Drive de los dos integrantes del grupo
file_path_dir = '../sandbox/football/kaggle/'
file_path_appearances = file_path_dir + 'appearances.csv'
file_path_players = file_path_dir + 'players.csv'
file_path_games = file_path_dir + 'games.csv'
file_path_player_valuations = file_path_dir + 'player_valuations.csv'

try:
  appearances = pd.read_csv(file_path_appearances)
  players = pd.read_csv(file_path_players)
  games = pd.read_csv(file_path_games)
  valuations = pd.read_csv(file_path_player_valuations)
  print("CSV loaded successfully.")
except FileNotFoundError:
  print("File not found at the specified path.")
except Exception as e:
  print("An error occurred while loading the CSV:", e)

CSV loaded successfully.


In [6]:
appearances

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45
3,2234418_73333,2234418,73333,1274,6646,2012-07-05,Vegar Hedenstad,ELQ,0,0,0,0,90
4,2234421_122011,2234421,122011,195,3008,2012-07-05,Markus Henriksen,ELQ,0,0,0,1,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589913,4346131_720287,4346131,720287,2741,2741,2024-06-02,Danila Kozlov,RUP,0,0,0,1,87
1589914,4346131_814269,4346131,814269,964,964,2024-06-02,Gustavo Mantuan,RUP,0,0,0,0,45
1589915,4346131_815346,4346131,815346,2741,2741,2024-06-02,Nathan Gassama,RUP,0,0,0,0,19
1589916,4346131_84483,4346131,84483,964,964,2024-06-02,Aleksandr Erokhin,RUP,1,0,0,0,26
